<a href="https://colab.research.google.com/github/Dhrubo0416/Data-Science-Cohort/blob/main/Project_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import json
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk import pos_tag
import numpy as np
import pickle
import string
import random


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer 

import warnings
warnings.simplefilter('ignore')

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
path = ("/content/drive/MyDrive/Chatbot project final/")
convdata = pd.read_csv(path+'Data_corpus_temp - Sheet1.csv')

#show header of the dataset
convdata.head()

,Questions,Answers
0,What are the job opportunities in government s...,There are many opportunities in government sec...
1,What was lowest package in production engineer...,"Lowest package stands between 4-5 LPA , it als..."
2,What are the prospects of Production Engineeri...,Production Engineering is a great choice of su...
3,Why everyone try for non core jobs is there no...,It is true that in core sectors opportunity is...
4,Is there any internship directly given by our ...,"Yes, many companies visit Campus for On Campus..."


In [ ]:
#covert dataframes to json
convdata_json = json.loads(convdata.to_json(orient='records'))
convdata_json[0:2]

[{'Questions': 'What are the job opportunities in government sector as a production engineer and the name of the companies ?',
  'Answers': 'There are many opportunities in government sectors if you get good rank in GATE ME(Mechanical Engineering) : \n1. Through GATE ME you can get job in PSU. ike : ONGC, NTPC, GAIL,\n2. You can be an IES officer through ESE.\n3 .You can be an ISRO,DRDO scientist (ISRO scientist exam ICRB is similar to GATE ME , DRDO scientist is similar to ESE )\n4. Your can be an government junior engineer through\xa0SSC\xa0JE\xa0exam'},
 {'Questions': 'What was lowest package in production engineering department of this year?',
  'Answers': 'Lowest package stands between 4-5 LPA , it also varies year to year.'}]

In [ ]:
#export as data as JSON
with open(path+'conversation_json.json', 'w') as outfile:
    json.dump(convdata_json, outfile)

In [ ]:
#greeting function
GREETING_INPUTS = ("hello", "hi", "greetings", "good morning", "good day","hey","i need help", "greetings")
GREETING_RESPONSES = ["Good day, How may i of help?", "Hello, How can i help?", "hello", "I am glad! You are talking to me."]
           
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [ ]:
#Wordnet Lemmatization 

lemmer = nltk.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

In [ ]:
# Remove punctuation
def RemovePunction(tokens):
    return[t for t in tokens if t not in string.punctuation]



In [ ]:
# Create a stopword list from the standard list of stopwords available in nltk
stop_words = set(stopwords.words('english'))
print(len(stop_words))
print(stop_words)

179
{'further', 'hadn', 're', 'was', 'too', 'over', 'doesn', 'during', 'whom', 'your', 'll', "won't", 'what', 'shan', 'has', "haven't", 'which', 'he', 'don', 'd', 'again', 'where', 'can', 'under', 'does', "don't", "aren't", 'until', 'not', "isn't", 'doing', 'being', 'but', 'her', 'while', "you're", 'out', 'isn', "mightn't", 'very', 'shouldn', 'are', 'a', 'won', 'only', 'having', 'we', "it's", 'hers', 'did', 'you', 'no', 'they', 'she', 'these', "needn't", 'such', 'm', 'himself', 'same', 's', 'had', 'o', 'as', 'their', 'in', 'other', 'there', 'between', 'some', 'most', 'than', "wouldn't", 'before', 'from', 'ain', 'haven', 'own', 'mightn', 'after', "weren't", 'if', 'hasn', 'here', "you'd", 'against', 'them', 'yours', 'off', 'me', 'by', 'because', 'its', 'and', 'at', "didn't", 'am', 'when', 'yourselves', "should've", 'be', 'been', 've', "shouldn't", 'each', "couldn't", 'all', 'above', 'theirs', 'it', 'wouldn', 'few', 'through', 'down', 'couldn', 't', 'were', "hasn't", 'below', 'herself', '

In [ ]:
def Talk_To_ProdBot(test_set_sentence):
    json_file_path = path+"conversation_json.json" 
    tfidf_vectorizer_pickle_path = path + "tfidf_vectorizer.pkl"
    tfidf_matrix_pickle_path = path+ "tfidf_matrix_train.pkl"
    
    i = 0
    sentences = []
    
    # ---------------Tokenisation of user input -----------------------------#
    
    tokens = RemovePunction(nltk.word_tokenize(test_set_sentence))
    pos_tokens = [word for word,pos in pos_tag(tokens, tagset='universal')]
    
    word_tokens = LemTokens(pos_tokens)
    
    filtered_sentence = []
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)  
    
    filtered_sentence =" ".join(filtered_sentence).lower()
            
    test_set = (filtered_sentence, "")
    
    #For Tracing, comment to remove from print.
    print('FILTERED INPUT->'+filtered_sentence)
    
    
    # -----------------------------------------------------------------------#
        
    try: 
        # ---------------Use Pre-Train Model------------------#
        f = open(tfidf_vectorizer_pickle_path, 'rb')
        tfidf_vectorizer = pickle.load(f)
        f.close()
        
        f = open(tfidf_matrix_pickle_path, 'rb')
        tfidf_matrix_train = pickle.load(f)
        f.close()
        # ---------------------------------------------------#
    except: 
        # ---------------To Train------------------#
        
        
        
        with open(json_file_path) as sentences_file:
            reader = json.load(sentences_file)
            
            # ---------------Tokenisation of training input -----------------------------#    
            
            for row in reader:
                db_tokens = RemovePunction(nltk.word_tokenize(row['Questions']))
                pos_db_tokens = [word for word,pos in pos_tag(db_tokens, tagset='universal')]
                db_word_tokens = LemTokens(pos_db_tokens)
                
                db_filtered_sentence = [] 
                for dbw in db_word_tokens: 
                    if dbw not in stop_words: 
                        db_filtered_sentence.append(dbw)  
                
                db_filtered_sentence =" ".join(db_filtered_sentence).lower()
                
                #Debugging Checkpoint
                print('TRAINING INPUT: '+db_filtered_sentence)
                
                sentences.append(db_filtered_sentence)
                i +=1                
            # ---------------------------------------------------------------------------#
                
        tfidf_vectorizer = TfidfVectorizer() 
        tfidf_matrix_train = tfidf_vectorizer.fit_transform(sentences)
        #print(tfidf_matrix_train)
    
        f = open(tfidf_vectorizer_pickle_path, 'wb')
        pickle.dump(tfidf_vectorizer, f) 
        f.close()
    
        f = open(tfidf_matrix_pickle_path, 'wb')
        pickle.dump(tfidf_matrix_train, f) 
        f.close()
        # ------------------------------------------#
        
    #use the learnt dimension space to run TF-IDF on the query
    tfidf_matrix_test = tfidf_vectorizer.transform(test_set)
    #print(tfidf_matrix_test)

    #then run cosine similarity between the 2 tf-idfs
    cosine = cosine_similarity(tfidf_matrix_test, tfidf_matrix_train)

    # print only observation and remove it
    #print(cosine.argsort())
    
    #if not in the topic trained.no similarity 
    idx= cosine.argsort()[0][-2]
    flat =  cosine.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    
    if (req_tfidf==0): #Threshold A

        cosine = np.delete(cosine, 0)
        not_understood = "Apology, I do not understand. Can you repeat it one more time elaborately?"
        
        return not_understood
        
    else:
        
        #print(cosine)
        cosine = np.delete(cosine, 0)
        #print(cosine)

        #get the max score
        max = cosine.max()
        response_index = 0

        #if max score is lower than < 0.5 > (we see can ask to rephrase.)
        if (max <= 0.5): #Threshold B
            
            not_understood = "Apology, I do not understand. Can you rephrase?"
            
            return not_understood
        else:

                # else we would simply return the highest score
                response_index = np.where(cosine == max)[0][0] + 2 

                j = 0 

                with open(json_file_path, "r") as sentences_file:
                    reader = json.load(sentences_file)
                    for row in reader:
                        j += 1 
                        if j == response_index: 
                            return row["Answers"]
                            break


In [ ]:
#pip install gradio

In [ ]:
#import gradio as gra 
#app =  gra.Interface(fn = Talk_To_ProdBot , inputs="text", outputs="text", allow_flagging="never")
#app.launch(share=True)

In [ ]:
flag=True
print("......................................................................................")
print( 'ProdBot'+': '+ 'I am  ProdBot,I will try my best to answer your query')
print('ProdBot'+':'+ ' If you want to exit, you can type < bye > .')
while(flag==True):
    print("......................................................................................")
    sentence = input("USER  "+":")
    print("......................................................................................")
    if(sentence.lower()!='bye'):
        if(greeting(sentence.lower())!=None):
            print('ProdBot'+': '+ greeting(sentence.lower()))
        else:
            response_primary = Talk_To_ProdBot(sentence)
            print('ProdBot'+': '+ response_primary)
            
    else:
        flag=False
print( 'ProdBot'+': '+"Bye! Hope that i am of help.") 

......................................................................................
ProdBot: I am  ProdBot,I will try my best to answer your query
ProdBot: If you want to exit, you can type < bye > .
......................................................................................
USER  :hi
......................................................................................
ProdBot: Hello, How can i help?
......................................................................................
USER  :what is the difference between production and mechanical engineering?
......................................................................................
FILTERED INPUT->difference production mechanical engineering
  (0, 253)	0.31758702750182943
  (0, 203)	0.5675881472289722
  (0, 105)	0.35394558703328627
  (0, 84)	0.6720897979492004
ProdBot: Mechanical Engineers focus more on product development works to design and manufacture new products for customers whereas Production Engine

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
